In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [13]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("citeseer")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.286026*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.18671026440138042


In [4]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 4,#trial.suggest_int("K", 2, 9),
        "layers": 1,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.18671026440138042,#trial.suggest_float("ratio", 0.12, 0.3),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": 'GCNConv',#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": 10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": 1024,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_val.csv",
        "min":0.82,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),
        "val":False         
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 20:59:47,759] A new study created in memory with name: no-name-5ef3b2dc-5b6e-4729-af8d-e575cbcebd66


Running experiment with seed=654:
 - K=4, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.18671026440138042, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 3.0996
Epoch 10 / 100, Loss: 2.5842
Epoch 20 / 100, Loss: 2.3823
Epoch 30 / 100, Loss: 2.1416
Epoch 40 / 100, Loss: 1.8704
Epoch 50 / 100, Loss: 1.6928
Epoch 60 / 100, Loss: 1.4348
Epoch 70 / 100, Loss: 1.2341
Epoch 80 / 100, Loss: 1.1190
Epoch 90 / 100, Loss: 0.9445
 - Test Metrics: Accuracy=0.9318, F1=0.8418, Recall=0.8616, Precision=0.8229
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=4, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.18671026440138042, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sag

[I 2025-04-14 21:04:57,647] Trial 0 finished with value: 0.8517810927645992 and parameters: {}. Best is trial 0 with value: 0.8517810927645992.


 - Test Metrics: Accuracy=0.9312, F1=0.8409, Recall=0.8631, Precision=0.8198
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to citeseer_experimentations\citeseer_sar_val_1404205947.csv.
Average F1 over valid seeds: 0.8518 ± 0.0111
Running experiment with seed=654:
 - K=4, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.18671026440138042, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 3.0996
Epoch 10 / 100, Loss: 2.5842


[W 2025-04-14 21:05:12,321] Trial 1 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_16804\3770530736.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 562, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 315, in train_graph
    global_ids_np = global_nids.cpu().numpy()
                    ^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-14 21:05:12,326] Trial 1 

KeyboardInterrupt: 

### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-25 10:10:43,640] A new study created in memory with name: no-name-f25fde7d-ee4a-4fa2-be4f-f9e486b8970e


Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=50, lr=0.0023234250431828636
Epoch 0 / 100, Loss: 2.8385
Epoch 10 / 100, Loss: 2.4108
Epoch 20 / 100, Loss: 2.2628
Epoch 30 / 100, Loss: 2.1825
Epoch 40 / 100, Loss: 2.0409
Epoch 50 / 100, Loss: 1.9945
Epoch 60 / 100, Loss: 1.8232
Epoch 70 / 100, Loss: 1.6971
Epoch 80 / 100, Loss: 1.6468
Epoch 90 / 100, Loss: 1.4848
 - Metrics: Accuracy=0.9405, F1=0.8507, Recall=0.8046, Precision=0.9024
Running experiment with seed=114:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pa

[I 2025-03-25 10:13:49,885] Trial 0 finished with value: 0.857595508989174 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.11307195638376069, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0023234250431828636, 'anomaly_detector': 'nearest_neighbors', 'clusters': 50, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.857595508989174.


 - Metrics: Accuracy=0.9390, F1=0.8493, Recall=0.8160, Precision=0.8854
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503101043.csv.
Average F1 over valid seeds: 0.8576 ± 0.0077
Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.17225386531043968, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=400, lr=0.008752505664905935
Epoch 0 / 150, Loss: 42.4167
Epoch 10 / 150, Loss: 18.6557
Epoch 20 / 150, Loss: 10.0401
Epoch 30 / 150, Loss: 6.0609
Epoch 40 / 150, Loss: 4.2251
Epoch 50 / 150, Loss: 3.0423
Epoch 60 / 150, Loss: 2.2833
Epoch 70 / 150, Loss: 1.8259
Epoch 80 / 150, Loss: 1.4801
Epoch 90 / 150, Loss: 1.2157
Epoch 100 / 150, Loss: 1.0309
Epoch 110 / 150, Loss: 0.8732
Epoch 120 / 150, Loss: 0.7590
Epoch 130 / 150, Loss: 0.6525
Epoch 140 / 150, Loss

[I 2025-03-25 10:15:40,677] Trial 1 finished with value: 0.6813996316758748 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.17225386531043968, 'model_type': 'GATConv', 'batch_size': 256, 'lr': 0.008752505664905935, 'anomaly_detector': 'unweighted', 'clusters': 400, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.857595508989174.


...

 - Metrics: Accuracy=0.9059, F1=0.7183, Recall=0.5692, Precision=0.9732
F1 = 0.72 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140907.csv.
Average F1 over valid seeds: 0.7183 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15841141409782936, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004733199465670689
Epoch 0 / 100, Loss: 2.1699
Epoch 10 / 100, Loss: 1.6532
Epoch 20 / 100, Loss: 1.4729
Epoch 30 / 100, Loss: 1.3237
Epoch 40 / 100, Loss: 1.1338
Epoch 50 / 100, Loss: 1.0336
Epoch 60 / 100, Loss: 0.8867
Epoch 70 / 100, Loss: 0.7703
Epoch 80 / 100, Loss: 0.7087
Epoch 90 / 100, Loss: 0.6093
 - Metrics: Accuracy=0.9408, F1=0.8600, Recall=0.8631, Precision=0.8569
Running experiment with seed=114:
 - K=2

[I 2025-03-25 14:12:40,226] Trial 98 finished with value: 0.8617602498928539 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.15841141409782936, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004733199465670689, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9387, F1=0.8539, Recall=0.8502, Precision=0.8576
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140941.csv.
Average F1 over valid seeds: 0.8618 ± 0.0066
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16102039029803536, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004605584968207975
Epoch 0 / 100, Loss: 4.1609
Epoch 10 / 100, Loss: 3.4473
Epoch 20 / 100, Loss: 3.0725
Epoch 30 / 100, Loss: 2.6641
Epoch 40 / 100, Loss: 2.3129
Epoch 50 / 100, Loss: 2.1049
Epoch 60 / 100, Loss: 1.7786
Epoch 70 / 100, Loss: 1.6623
Epoch 80 / 100, Loss: 1.4632
Epoch 90 / 100, Loss: 1.3323


[I 2025-03-25 14:13:15,258] Trial 99 finished with value: 0.8429629629629629 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16102039029803536, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004605584968207975, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9363, F1=0.8430, Recall=0.8117, Precision=0.8767
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503141240.csv.
Average F1 over valid seeds: 0.8430 ± 0.0000
Best trial:
  Average F1: 0.8650138572197479
  Best parameters:
    K: 2
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.1611386183555754
    model_type: GCNConv
    batch_size: 1024
    lr: 0.0063212510924867535
    anomaly_detector: nearest_neighbors
    clusters: 200
    num_epochs: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SCAR 

In [5]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("cora")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.379923*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.24134569855135768


In [7]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 4,#trial.suggest_int("K", 1, 12),
        "layers": 1,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 64,# trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.24134569855135768,#trial.suggest_float("ratio", 0.16, 0.38),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size":1024,# trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False,    
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 21:19:29,759] A new study created in memory with name: no-name-01d599ce-7764-4ead-a7e6-dc20f15cbab5


Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 2.6197
Epoch 10 / 100, Loss: 2.2504
Epoch 20 / 100, Loss: 2.0526
Epoch 30 / 100, Loss: 1.9024
Epoch 40 / 100, Loss: 1.7537
Epoch 50 / 100, Loss: 1.5948
Epoch 60 / 100, Loss: 1.4321
Epoch 70 / 100, Loss: 1.2943
Epoch 80 / 100, Loss: 1.1744
Epoch 90 / 100, Loss: 1.0556
 - Test Metrics: Accuracy=0.9313, F1=0.8885, Recall=0.9059, Precision=0.8718
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage


[I 2025-04-14 21:22:44,695] Trial 0 finished with value: 0.8945582325243089 and parameters: {'sampling': 'sage'}. Best is trial 0 with value: 0.8945582325243089.


 - Test Metrics: Accuracy=0.9324, F1=0.8892, Recall=0.8973, Precision=0.8812
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to cora_experimentations\cora_sar_1404211929.csv.
Average F1 over valid seeds: 0.8946 ± 0.0052
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 2.6197
Epoch 10 / 100, Loss: 2.2504
Epoch 20 / 100, Loss: 2.0526
Epoch 30 / 100, Loss: 1.9024


[W 2025-04-14 21:23:02,758] Trial 1 failed with parameters: {'sampling': 'sage'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_16804\4126520310.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 562, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 333, in train_graph
    scaler.step(optimizer)
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\torch\amp\grad_scaler.py",

KeyboardInterrupt: 

### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [8]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("pubmed")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.489441*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.31766718352866136


In [16]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 12,#trial.suggest_int("K", 2, 20),
        "layers":2,# trial.suggest_int("layers", 1, 2),
        "hidden_channels":256,# trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.31766718352866136,#trial.suggest_float("ratio", 0.2, 0.5),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": 'GCNConv',#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scr_val.csv",
        "min":0.83,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False    
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:20:59,665] A new study created in memory with name: no-name-16d3bc65-343e-48ef-9577-ee4430a12662


Running experiment with seed=654:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.31766718352866136, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 16.4457
Epoch 10 / 100, Loss: 13.9870
Epoch 20 / 100, Loss: 10.9916
Epoch 30 / 100, Loss: 7.8874
Epoch 40 / 100, Loss: 5.5521
Epoch 50 / 100, Loss: 3.9770
Epoch 60 / 100, Loss: 2.9568
Epoch 70 / 100, Loss: 2.2980
Epoch 80 / 100, Loss: 1.8337
Epoch 90 / 100, Loss: 1.5076
 - Test Metrics: Accuracy=0.8933, F1=0.8671, Recall=0.8714, Precision=0.8629
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.31766718352866136, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, samplin

[I 2025-04-14 22:45:28,393] Trial 0 finished with value: 0.8662528484137495 and parameters: {}. Best is trial 0 with value: 0.8662528484137495.


 - Test Metrics: Accuracy=0.8870, F1=0.8588, Recall=0.8606, Precision=0.8570
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to pubmed_experimentations\pubmed_scr_val_1404222059.csv.
Average F1 over valid seeds: 0.8663 ± 0.0042
Best trial:
  Average F1: 0.8662528484137495
  Best parameters:


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [12]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("wiki-cs")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.291999*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.1822305564160316


In [15]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": 3,#trial.suggest_int("K", 2, 35),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.1822305564160316,#trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:11:10,847] A new study created in memory with name: no-name-1f95e279-aada-49a9-a5d2-2cdb69773da7


Running experiment with seed=654:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.6099
Epoch 10 / 100, Loss: 3.8026
Epoch 20 / 100, Loss: 3.1336
Epoch 30 / 100, Loss: 2.5338
Epoch 40 / 100, Loss: 2.0711
Epoch 50 / 100, Loss: 1.7149
Epoch 60 / 100, Loss: 1.4327
Epoch 70 / 100, Loss: 1.2042
Epoch 80 / 100, Loss: 1.0330
Epoch 90 / 100, Loss: 0.8866
 - Test Metrics: Accuracy=0.9615, F1=0.9182, Recall=0.9425, Precision=0.8951
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 -

[I 2025-04-14 22:20:59,610] Trial 0 finished with value: 0.9146052245003616 and parameters: {}. Best is trial 0 with value: 0.9146052245003616.


 - Test Metrics: Accuracy=0.9592, F1=0.9127, Recall=0.9302, Precision=0.8958
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1404221110.csv.
Average F1 over valid seeds: 0.9146 ± 0.0020
Best trial:
  Average F1: 0.9146052245003616
  Best parameters:


### Hyperparameter Optimization WikiCS
#### SAR 

In [14]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 3,#trial.suggest_int("K", 2, 35),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.1822305564160316,#trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:00:29,047] A new study created in memory with name: no-name-1a4f9b7d-322b-4c27-bab1-ba7a9f3fd599


Running experiment with seed=654:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.5456
Epoch 10 / 100, Loss: 3.7924
Epoch 20 / 100, Loss: 3.0968
Epoch 30 / 100, Loss: 2.5100
Epoch 40 / 100, Loss: 2.0480
Epoch 50 / 100, Loss: 1.6982
Epoch 60 / 100, Loss: 1.4190
Epoch 70 / 100, Loss: 1.1968
Epoch 80 / 100, Loss: 1.0250
Epoch 90 / 100, Loss: 0.8854
 - Test Metrics: Accuracy=0.9599, F1=0.9145, Recall=0.9362, Precision=0.8938
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 -

[I 2025-04-14 22:11:10,444] Trial 0 finished with value: 0.9125326523955319 and parameters: {}. Best is trial 0 with value: 0.9125326523955319.


 - Test Metrics: Accuracy=0.9564, F1=0.9070, Recall=0.9287, Precision=0.8864
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to wiki-cs_experimentations\wikics_sar_val_1404220029.csv.
Average F1 over valid seeds: 0.9125 ± 0.0028
Best trial:
  Average F1: 0.9125326523955319
  Best parameters:


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,#trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [2048,4096,8192]),
        "lr": 0.005,#trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.6,
        "clusters":50,#trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":100,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"sage",#trial.suggest_categorical("sampling",["neighbor","cluster"]),   
        "val":True      
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:45:28,445] A new study created in memory with name: no-name-6260d805-ad64-4856-b049-9412104495ff


Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=8192, methodology=ours
 - ratio=0.02908978668035898, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 28.1691
Epoch 10 / 100, Loss: 13.3677
Epoch 20 / 100, Loss: 6.3969
Epoch 30 / 100, Loss: 3.6775
Epoch 40 / 100, Loss: 2.3237
Epoch 50 / 100, Loss: 1.6122
Epoch 60 / 100, Loss: 1.1822
Epoch 70 / 100, Loss: 0.8975
Epoch 80 / 100, Loss: 0.7016
Epoch 90 / 100, Loss: 0.5626


[I 2025-04-14 23:31:58,459] Trial 0 finished with value: 0.2332814930015552 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.02908978668035898, 'model_type': 'GCNConv', 'batch_size': 8192}. Best is trial 0 with value: 0.2332814930015552.


 - Test Metrics: Accuracy=0.9381, F1=0.5538, Recall=0.3934, Precision=0.9351
 - Validation Metrics: Accuracy=0.1320, F1=0.2333, Recall=0.1320, Precision=1.0000
F1 = 0.23 < 0.6, skipping ...
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1404224528.csv.
Average F1 over valid seeds: 0.2333 ± 0.0000
Running experiment with seed=654:
 - K=6, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.02183855894866076, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 134.9077
Epoch 10 / 100, Loss: 12.7824
Epoch 20 / 100, Loss: 3.9007
Epoch 30 / 100, Loss: 1.7160
Epoch 40 / 100, Loss: 0.8764
Epoch 50 / 100, Loss: 0.4827
Epoch 60 / 100, Loss: 0.2796
Epoch 70 / 100, Loss: 0.1639
Epoch 80 / 100, Loss: 0.0979
Epoch 90 / 100, Loss: 0.0588


[I 2025-04-15 00:19:33,955] Trial 1 finished with value: 0.34643377001455605 and parameters: {'K': 6, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'ratio': 0.02183855894866076, 'model_type': 'GATConv', 'batch_size': 2048}. Best is trial 1 with value: 0.34643377001455605.


 - Test Metrics: Accuracy=0.9361, F1=0.5623, Recall=0.4202, Precision=0.8493
 - Validation Metrics: Accuracy=0.2095, F1=0.3464, Recall=0.2095, Precision=1.0000
F1 = 0.35 < 0.6, skipping ...
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1404233158.csv.
Average F1 over valid seeds: 0.3464 ± 0.0000
Running experiment with seed=654:
 - K=19, layers=1, hidden=256, out=128
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.021152053625931862, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 326.2339
Epoch 10 / 100, Loss: 44.0523
Epoch 20 / 100, Loss: 12.0608
Epoch 30 / 100, Loss: 4.9084
Epoch 40 / 100, Loss: 2.4459
Epoch 50 / 100, Loss: 1.3367
Epoch 60 / 100, Loss: 0.7631
Epoch 70 / 100, Loss: 0.4467
Epoch 80 / 100, Loss: 0.2658
Epoch 90 / 100, Loss: 0.1600


[I 2025-04-15 01:15:45,969] Trial 2 finished with value: 0.1437908496732026 and parameters: {'K': 19, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.021152053625931862, 'model_type': 'GATConv', 'batch_size': 2048}. Best is trial 1 with value: 0.34643377001455605.


 - Test Metrics: Accuracy=0.9312, F1=0.5185, Recall=0.3793, Precision=0.8190
 - Validation Metrics: Accuracy=0.0775, F1=0.1438, Recall=0.0775, Precision=1.0000
F1 = 0.14 < 0.6, skipping ...
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1504001933.csv.
Average F1 over valid seeds: 0.1438 ± 0.0000
Running experiment with seed=654:
 - K=19, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=8192, methodology=ours
 - ratio=0.019854897067080417, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 79.3364
Epoch 10 / 100, Loss: 41.9610
Epoch 20 / 100, Loss: 24.8372
Epoch 30 / 100, Loss: 15.0954
Epoch 40 / 100, Loss: 9.3570
Epoch 50 / 100, Loss: 6.1353
Epoch 60 / 100, Loss: 4.2811
Epoch 70 / 100, Loss: 3.1357
Epoch 80 / 100, Loss: 2.3869
Epoch 90 / 100, Loss: 1.8633


[I 2025-04-15 02:32:08,374] Trial 3 finished with value: 0.18819776714513556 and parameters: {'K': 19, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.019854897067080417, 'model_type': 'GATConv', 'batch_size': 8192}. Best is trial 1 with value: 0.34643377001455605.


 - Test Metrics: Accuracy=0.9347, F1=0.5382, Recall=0.3897, Precision=0.8698
 - Validation Metrics: Accuracy=0.1039, F1=0.1882, Recall=0.1039, Precision=1.0000
F1 = 0.19 < 0.6, skipping ...
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1504011545.csv.
Average F1 over valid seeds: 0.1882 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=8192, methodology=ours
 - ratio=0.008261270254095305, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 23.1630
Epoch 10 / 100, Loss: 10.5033
Epoch 20 / 100, Loss: 5.0795
Epoch 30 / 100, Loss: 2.9233
Epoch 40 / 100, Loss: 1.9042
Epoch 50 / 100, Loss: 1.3304
